In [2]:
import json
import re

input_file = '../../iclr-discourse-dataset/review_rebuttal_pair_dataset_debug/traindev_train_clean.json'
with open(input_file) as f:
  data = json.load(f)

all_pairs = data['review_rebuttal_pairs']
# print(all_pairs[0].keys()) # output: dict_keys(['index', 'review_sid', 'rebuttal_sid', 'review_text', 'rebuttal_text', 'title', 'review_author', 'forum', 'labels'])

example_reviews = []
for current_pair in all_pairs:
    # if current_pair['forum'] == 'H1e0-30qKm':
    current_review = {}
    current_review['review_text'] = current_pair['review_text']
    current_review['title'] = current_pair['title']
    current_review['review_author'] = current_pair['review_author']
    current_review['forum'] = current_pair['forum']
    current_review['labels'] = current_pair['labels']
    current_review['pros'] = []
    current_review['cons'] = []
    # current_review['minor'] = [] # other minor points, e.g., refs, typos
    example_reviews.append(current_review)
    # print(current_pair['index'], current_pair['forum'], current_pair['title'])

for review in example_reviews:
    review_text = review['review_text']
    for paragraph_index, paragraph in enumerate(review_text):
        if is_special_paragraph(paragraph):
            process_paragraph(paragraph, paragraph_index, review_text, review)
    print(review['title'])
    print('PRO', len(review['pros']), review['pros'])
    print('CON', len(review['cons']), review['cons'])

Unlabeled Disentangling of GANs with Guided Siamese Networks
PRO 0 []
CON 0 []
Unlabeled Disentangling of GANs with Guided Siamese Networks
PRO 0 []
CON 0 []
Unlabeled Disentangling of GANs with Guided Siamese Networks
PRO 0 []
CON 0 []
Unlabeled Disentangling of GANs with Guided Siamese Networks
PRO 0 []
CON 0 []
Heated-Up Softmax Embedding
PRO 0 []
CON 0 []
Heated-Up Softmax Embedding
PRO 0 []
CON 0 []
Heated-Up Softmax Embedding
PRO 2 [[['+'], ['The', 'idea', 'of', 'heating', 'up', 'the', 'temperature', 'in', 'softmax', 'is', 'interesting', ',', 'and', 'seems', 'novel', 'in', 'the', 'literature', 'of', 'metric', 'learning', '.']], [['+'], ['The', 'performance', 'improvement', ',', 'especially', 'produced', 'by', 'batchNorm', '-', 'based', 'normalization', ',', 'is', 'shown', '.']]]
CON 4 [[['-'], ['The', 'formulation', 'of', 'tempered', 'softmax', 'with', 'normalization', 'is', 'already', 'presented', 'in', '[', 'Wang', 'et', 'al.', ',', '2017', ']', '.']], [['-'], ['The', 'reason',

In [1]:
word_in_sentence_cutoff = 5 # minimal number of words that a regular sentence should have, in order to exclude special paragraphs, e.g., formatting like '=====', headings like 'Summary:'

# paragraph contains one sentence with less than word_in_sentence_cutoff words
# start with '[', e.g., [1] citation
# end with ':'
def is_special_paragraph(paragraph):
    if len(paragraph) == 1 and len(paragraph[0]) < word_in_sentence_cutoff or paragraph[0][0] == '[' or paragraph[len(paragraph)-1][len(paragraph[len(paragraph)-1])-1] == ':':
        return True
    return False

def process_paragraph(paragraph, paragraph_index, review_text, review):
    if len(paragraph[0]) < word_in_sentence_cutoff:
        first_word = paragraph[0][0]
        if re.match('pro', first_word, re.I):
            # print(review['title'])
            paragraph_index += 1
            bullet = review_text[paragraph_index][0][0]
            if re.match('[a-z]', bullet, re.I):
                # add each paragraph as a point
                for index in range(paragraph_index, len(review_text)):
                    if not is_special_paragraph(review_text[index]):
                        review['pros'].append(review_text[index])
                    else:
                        break # if the bullet changes, it's the end of pros
            elif re.match('[0-9]', bullet):
                for index in range(paragraph_index, len(review_text)):
                    if re.match('[0-9]', review_text[index][0][0]):
                        review['pros'].append(review_text[index])
                    else:
                        break # if the bullet changes, it's the end of pros
            else:
                for index in range(paragraph_index, len(review_text)):
                    if review_text[index][0][0] == bullet:
                        review['pros'].append(review_text[index])
                    else:
                        break # if the bullet changes, it's the end of pros

        if re.match('con', first_word, re.I):
            # print(review['title'])
            paragraph_index += 1
            bullet = review_text[paragraph_index][0][0]
            if re.match('[a-z]', bullet, re.I):
                # add each paragraph as a point
                for index in range(paragraph_index, len(review_text)):
                    if not is_special_paragraph(review_text[index]):
                        review['cons'].append(review_text[index])
                    else:
                        break # if the bullet changes, it's the end of cons
            elif re.match('[0-9]', bullet):
                for index in range(paragraph_index, len(review_text)):
                    if re.match('[0-9]', review_text[index][0][0]):
                        review['cons'].append(review_text[index])
                    else:
                        break # if the bullet changes, it's the end of cons
            else:
                for index in range(paragraph_index, len(review_text)):
                    if review_text[index][0][0] == bullet:
                        review['cons'].append(review_text[index])
                    else:
                        break # if the bullet changes, it's the end of cons
